In [1]:
import sys, os, git, time

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from InitializeSimulation import InitializeSimulation
from Utilities import Utilities
from Outputs import Outputs
from MolecularDynamics import MolecularDynamics
from MonteCarlo import MonteCarlo

import numpy as np

In [24]:
self = MolecularDynamics(number_atoms=[50, 50],
                      epsilon=[0.1, 0.1], # kcal/mol
                      sigma=[2, 1], # A
                      atom_mass=[1, 1], # g/mol
                      atom_charge=[1,-1], # in elementary charge units
                      Lx=30, # A
                      Ly=30, # A
                      Lz=30, # A
                      minimization_steps = 50,
                      maximum_steps=500,
                      desired_temperature=300,
                      thermo = 10,
                      dump = 10,  
                      tau_temp = 100, # fs
                      #tau_press= 1000, # fs
                      time_step=1, # fs
                      seed=319817,
                      data_folder = "mdcode-output/",
                      )
self.run()
# run lammps for comparison
os.system("/home/simon/Softwares/lammps-2Aug2023/src/lmp_serial -in input.lammps")

step epot maxF
0 141616.186 5345485.986
5 856.952 36173.248
10 -0.368 14.793
15 -1.344 0.180
20 -1.511 0.202
25 -1.610 0.193
30 -1.674 0.231
35 -1.815 0.191
40 -1.913 0.226
45 -1.977 0.165
50 -1.999 0.085
 step     N  temp  epot  ekin press   vol
    0   100 299.97054616262244 -1.9907489036159403 88.52125789997515 148.18015040958107 27000.0
   10   100 295.98876436562125 -0.7695100948384294 87.34623475899423 159.16011006045377 27000.0
   20   100 298.0771577353297 -1.279773516207083 87.96251929239753 150.95415287209752 27000.0
   30   100 298.3387709804332 -1.3053958188088093 88.03972131718331 146.853146246287 27000.0
   40   100 298.39133045035004 -1.2719609178086433 88.05523160794723 145.22974671542184 27000.0
   50   100 297.40569082783094 -0.9284264427947874 87.764369520527 149.13343630599783 27000.0
   60   100 296.4944503598689 -0.5328846620236388 87.49546261114786 153.6628697048578 27000.0
   70   100 297.42046963318967 -0.7324763224830297 87.76873074351155 150.54010658908425 27

0

In [26]:
from itertools import combinations
from scipy.special import erfc # complementary error function : ercf = 1 - erf
import numpy as np

In [27]:
r = self.atoms_positions
n = 4
pot = 0
kappa = 6
for position_i, charge_i in zip(self.atoms_positions, self.atoms_charge):
    positions_j = self.atoms_positions
    charges_j = self.atoms_charge
    rij_xyz = (np.remainder(position_i - positions_j
        + self.box_size/2., self.box_size) - self.box_size/2.)
    norm_rij = np.linalg.norm(rij_xyz, axis=1)
    vij = charge_i * charges_j[norm_rij>0] * erfc ( kappa * norm_rij[norm_rij>0] ) / norm_rij[norm_rij>0] # Screened Coulomb term
    pot += np.sum(vij)
print(pot)

3.249494500532814e-08


In [22]:
pot

-1.338583312689866e-23

In [18]:
charge_i

-1

In [9]:

for i,j in combinations(range(len(r)),2):
    rij  = r[i,:] - r[j,:]            # Separation vector (box=1 units)
    rij  = rij - np.rint ( rij )      # Periodic boundaries
    rij_mag = np.sqrt(np.sum(rij**2)) # Magnitude of separation
    vij = q[i] * q[j] * erfc ( kappa * rij_mag ) / rij_mag # Screened Coulomb term

NameError: name 'q' is not defined

In [15]:
rij_mag

0.5415847123520906

In [1]:
pot = 0.0
for i,j in combinations(range(n),2):  # Double loop over pairs of atoms
    rij  = r[i,:] - r[j,:]            # Separation vector (box=1 units)
    rij  = rij - np.rint ( rij )      # Periodic boundaries
    rij_mag = np.sqrt(np.sum(rij**2)) # Magnitude of separation
    vij = q[i] * q[j] * erfc ( kappa * rij_mag ) / rij_mag # Screened Coulomb term
    pot = pot + vij

NameError: name 'n' is not defined